In [0]:
import requests
import json
from datetime import datetime, UTC

ingest_date = datetime.now(UTC).strftime("%Y-%m-%d")

print(f"Ingest date: {ingest_date}")

url = "https://api.openbrewerydb.org/v1/breweries"

# ADLS Bronze Raw Path (JSON Lines)
bronze_raw_dir = f"/mnt/datalake/bronze/breweries/date={ingest_date}"
bronze_raw_file = f"{bronze_raw_dir}/raw.json"

print(f"Saving to: {bronze_raw_file}")

# API
response = requests.get(url, timeout=30)
response.raise_for_status()

data = response.json()  # list of dictionaries

print(f"Downloaded {len(data)} brewery records")

# 4. Save raw JSON as JSON Lines (Bronze best practice)
dbutils.fs.mkdirs(bronze_raw_dir)  # create folder if not exists

json_lines = "\n".join(json.dumps(row) for row in data)

dbutils.fs.put(
    bronze_raw_file,
    json_lines,
    overwrite=True
)

print("Bronze layer completed")


Ingest date: 2025-11-23
Saving to: /mnt/datalake/bronze/breweries/date=2025-11-23/raw.json
Downloaded 50 brewery records
Wrote 22185 bytes.
Bronze layer completed


In [0]:
df_raw = spark.read.json("/mnt/datalake/bronze/breweries")
display(df_raw)

address_1,address_2,address_3,brewery_type,city,country,id,latitude,longitude,name,phone,postal_code,state,state_province,street,website_url,date
1716 Topeka St,null,null,micro,Norman,United States,5128df48-79fc-4f0f-8b52-d06be54d0cec,35.25738891,-97.46818222,(405) Brewing Co,4058160490,73069-8224,Oklahoma,Oklahoma,1716 Topeka St,http://www.405brewing.com,2025-11-23
407 Radam Ln Ste F200,null,null,micro,Austin,United States,9c5a66c8-cc13-416f-a5d9-0a769c87d318,null,null,(512) Brewing Co,5129211545,78745-1197,Texas,Texas,407 Radam Ln Ste F200,http://www.512brewing.com,2025-11-23
8100 Washington Ave,null,null,micro,Mount Pleasant,United States,34e8c68b-6146-453f-a4b9-1f6cd99a5ada,42.720108268996,-87.883363502094,1 of Us Brewing Company,2624847553,53406-3920,Wisconsin,Wisconsin,8100 Washington Ave,https://www.1ofusbrewing.com,2025-11-23
62970 18th St,null,null,large,Bend,United States,6d14b220-8926-4521-8d19-b98a2d6ec3db,44.08683531,-121.281706,10 Barrel Brewing Co,5415851007,97701-9847,Oregon,Oregon,62970 18th St,http://www.10barrel.com,2025-11-23
1135 NW Galveston Ave Ste B,null,null,large,Bend,United States,e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2,44.0575649,-121.3288021,10 Barrel Brewing Co,5415851007,97703-2465,Oregon,Oregon,1135 NW Galveston Ave Ste B,null,2025-11-23
1411 NW Flanders St,null,null,large,Portland,United States,e432899b-7f58-455f-9c7b-9a6e2130a1e0,45.5259786,-122.6855056,10 Barrel Brewing Co,5032241700,97209-2620,Oregon,Oregon,1411 NW Flanders St,http://www.10barrel.com,2025-11-23
1501 E St,null,null,large,San Diego,United States,ef970757-fe42-416f-931d-722451f1f59c,32.714813,-117.129593,10 Barrel Brewing Co,6195782311,92101-6618,California,California,1501 E St,http://10barrel.com,2025-11-23
62950 NE 18th St,null,null,large,Bend,United States,9f1852da-c312-42da-9a31-097bac81c4c0,44.0912109,-121.2809536,10 Barrel Brewing Co - Bend Pub,5415851007,97701,Oregon,Oregon,62950 NE 18th St,null,2025-11-23
826 W Bannock St,null,null,large,Boise,United States,ea4f30c0-bce6-416b-8904-fab4055a7362,43.618516,-116.202929,10 Barrel Brewing Co - Boise,2083445870,83702-5857,Idaho,Idaho,826 W Bannock St,http://www.10barrel.com,2025-11-23
2620 Walnut St,null,null,large,Denver,United States,1988eb86-f0a2-4674-ba04-02454efa0d31,39.7592508,-104.9853655,10 Barrel Brewing Co - Denver,7205738992,80205-2231,Colorado,Colorado,2620 Walnut St,null,2025-11-23


In [0]:
%fs
ls /mnt/datalake/bronze/breweries/

path,name,size,modificationTime
dbfs:/mnt/datalake/bronze/breweries/date=2025-11-23/,date=2025-11-23/,0,1763857056000
